In [1]:
import pandas as pd
import os
import geopandas as gpd
cwd = os.getcwd()
from utils import rename_columns,filter_by_year,filter_columns,process_data
from intersect_points import select_intersection_points

In [2]:
points = select_intersection_points('../data/DHareas/DH_areas_plandata_DK.geojson', '../data/Varmeplan Danmark 2021/industrial_excess_heat_new_classes.geojson')
points.to_file('../data/Varmeplan Danmark 2021/points_in_DH.geojson')

In [5]:
points = gpd.read_file('../data/Varmeplan Danmark 2021/points_in_DH.geojson')

In [ ]:
print(points['GJ_over_80'].sum(), gdf['GJ_over_80'].sum())

In [6]:
# configure the data
renamed_columns_list = {
    "vaerk_ID": "plant_id",
    "aar": "year",
    "anlaegstype_navn": "plant_type",
    "vaerk_kommune": "municipality_code",
    "varmeprod_TJ": "heat_production_TJ"
}

relevant_columns =['plant_id', 'year','municipality_code', 'heat_production_TJ']
year=2022
# Read the data
df_raw = pd.read_excel('../data/EPT_DEA/6_ept_produktions-_og_forbrugsdata_2021-2023.xlsx', sheet_name='EPT2021-2023')
df_raw= rename_columns(df_raw,renamed_columns_list)
df_raw = filter_columns(df_raw,relevant_columns)
df = df_raw.copy()
df = filter_by_year(df,year)

# Calculate the total heat production per municipality (and year)
df['total_municipality_heat_production_TJ'] = (
    df.groupby(['year', 'municipality_code'])['heat_production_TJ']
        .transform('sum')
)

In [8]:
df['total_municipality_heat_production_GJ'] = df['total_municipality_heat_production_TJ']*1000

In [9]:
# Group by year, municipality_code, and plant_type to sum heat_production_TJ
potentials_by_municipality = (
    points.groupby([ 'kommuneKod'], as_index=False).agg(
      {'GJ_samlet':'sum', 'GJ_over_80':'sum', 'GJ_60_80C':'sum', 'GJ_under_6':'sum'} )
)

share_potentials_by_municipality = df [['municipality_code', 'total_municipality_heat_production_TJ','total_municipality_heat_production_GJ']].merge(
         potentials_by_municipality,
                         right_on='kommuneKod', left_on='municipality_code')

share_potentials_by_municipality['totalpot_share_of_total_heat_production_%'] = share_potentials_by_municipality['GJ_samlet'] * 100/ share_potentials_by_municipality['total_municipality_heat_production_GJ']
share_potentials_by_municipality['ht_share_of_total_heat_production_%'] = share_potentials_by_municipality['GJ_over_80'] * 100/ share_potentials_by_municipality['total_municipality_heat_production_GJ']
share_potentials_by_municipality['mt_share_of_total_heat_production_%'] = share_potentials_by_municipality['GJ_60_80C'] * 100/ share_potentials_by_municipality['total_municipality_heat_production_GJ']
share_potentials_by_municipality['lt_share_of_total_heat_production_%'] = share_potentials_by_municipality['GJ_under_6'] * 100/ share_potentials_by_municipality['total_municipality_heat_production_GJ']

In [10]:
# read the geojson file
municipalities_gdf = gpd.read_file('../data/kommuner/kommuner.geojson')
municipalities_gdf.drop(columns=['year'], inplace=True)

In [11]:
gdf_joined = share_potentials_by_municipality.merge(municipalities_gdf, right_on='municipality_code', left_on= 'municipality_code')
gdf_joined = gdf_joined.drop_duplicates()
gdf_joined = gpd.GeoDataFrame(gdf_joined, geometry='geometry')
os.makedirs('../data/processed', exist_ok=True)
gdf_joined.to_file(f'../data/processed/municipality_varmeplan_potentials.geojson', driver='GeoJSON')